# Local Modeling

## Libraries to Import

In [2]:
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

from PIL import Image
from tensorflow.keras.preprocessing import image, image_dataset_from_directory
from tensorflow.keras import models, layers, optimizers
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import BatchNormalization

import os
import gc
from timeit import default_timer as timer

## Define Image Directories

In [3]:
train_dir = '/Users/arencarpenter/Desktop/Detecting_ALL_with_CNN/Data/training/'
test_dir = '/Users/arencarpenter/Desktop/Detecting_ALL_with_CNN/Data/testing/'
val_dir = '/Users/arencarpenter/Desktop/Detecting_ALL_with_CNN/Data/validation/'

Create flow_from_directory objects for Keras model. Using a downsized dataset with class balance for local model running.

In [4]:
train_g = image.ImageDataGenerator(rescale = 1/255).flow_from_directory(train_dir,
                                                                  target_size = (256,256), 
                                                                  color_mode='rgb',
                                                                  class_mode='binary')
val_g = image.ImageDataGenerator(rescale = 1/255).flow_from_directory(val_dir,
                                                                target_size = (256,256), 
                                                                color_mode='rgb',
                                                                class_mode='binary')
test_g = image.ImageDataGenerator(rescale = 1/255).flow_from_directory(test_dir,
                                                                target_size = (256,256), 
                                                                color_mode='rgb',
                                                                class_mode='binary')

Found 1346 images belonging to 2 classes.
Found 593 images belonging to 2 classes.
Found 947 images belonging to 2 classes.


## Vanilla Model - 1C1D

In [4]:
model = models.Sequential()
model.add(layers.Conv2D(64, 3, activation = 'relu', input_shape=(256,256,3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation = 'relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [5]:
# compiling models
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['accuracy', 'Recall'])

In [6]:
# training
start = timer()

history = model.fit(train_g, 
                    epochs=5, 
                    validation_data=val_g, 
                    workers = 7
                   )

end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/5
43/43 [==============================] - 410s 10s/step - loss: 1.0253 - accuracy: 0.6478 - recall: 0.6419 - val_loss: 0.6186 - val_accuracy: 0.7386 - val_recall: 0.5612
Epoch 2/5
43/43 [==============================] - 378s 9s/step - loss: 0.6211 - accuracy: 0.7259 - recall: 0.7073 - val_loss: 0.5100 - val_accuracy: 0.7639 - val_recall: 0.8265
Epoch 3/5
43/43 [==============================] - 308s 7s/step - loss: 0.4892 - accuracy: 0.7749 - recall: 0.7816 - val_loss: 0.6214 - val_accuracy: 0.7150 - val_recall: 0.8980
Epoch 4/5
43/43 [==============================] - 306s 7s/step - loss: 0.4327 - accuracy: 0.8061 - recall: 0.8009 - val_loss: 0.7740 - val_accuracy: 0.6358 - val_recall: 0.2721
Epoch 5/5
43/43 [==============================] - 346s 8s/step - loss: 0.4165 - accuracy: 0.8068 - recall: 0.7875 - val_loss: 0.5517 - val_accuracy: 0.7808 - val_recall: 0.7007
Total Time Elapsed:  29  minutes  56  seconds


In [7]:
scores = model.evaluate(test_g, verbose=1)

30/30 [==============================] - 14s 474ms/step - loss: 1.2477 - accuracy: 0.5164 - recall: 0.3272


## 2C1D

Add another layer.

In [4]:
model = models.Sequential()
model.add(layers.Conv2D(64, 3, activation = 'relu', input_shape=(256,256,3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['accuracy', 'Recall'])

start = timer()

history = model.fit(train_g, 
                    epochs=5, 
                    validation_data=val_g, 
                    workers = 7
                   )

end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/5
43/43 [==============================] - 100s 2s/step - loss: 0.5483 - accuracy: 0.7385 - recall: 0.6909 - val_loss: 0.5160 - val_accuracy: 0.7622 - val_recall: 0.7823
Epoch 2/5
43/43 [==============================] - 103s 2s/step - loss: 0.5017 - accuracy: 0.7697 - recall: 0.7311 - val_loss: 0.5093 - val_accuracy: 0.7656 - val_recall: 0.8061
Epoch 3/5
43/43 [==============================] - 100s 2s/step - loss: 0.4607 - accuracy: 0.7838 - recall: 0.7340 - val_loss: 0.4995 - val_accuracy: 0.7707 - val_recall: 0.7823
Epoch 4/5
43/43 [==============================] - 86s 2s/step - loss: 0.4479 - accuracy: 0.7987 - recall: 0.7756 - val_loss: 0.5273 - val_accuracy: 0.7589 - val_recall: 0.8435
Epoch 5/5
43/43 [==============================] - 92s 2s/step - loss: 0.4196 - accuracy: 0.8039 - recall: 0.7860 - val_loss: 0.5227 - val_accuracy: 0.7673 - val_recall: 0.8265
Total Time Elapsed:  8  minutes  15  seconds


In [5]:
scores = model.evaluate(test_g, verbose=1)

30/30 [==============================] - 12s 393ms/step - loss: 0.8072 - accuracy: 0.5744 - recall: 0.4676


## 3C1D

Add another layer.

In [6]:
model = models.Sequential()
model.add(layers.Conv2D(64, 3, activation = 'relu', input_shape=(256,256,3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['accuracy', 'Recall'])

start = timer()

history = model.fit(train_g, 
                    epochs=5, 
                    validation_data=val_g, 
                    workers = 7
                   )

end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/5
43/43 [==============================] - 76s 2s/step - loss: 0.5603 - accuracy: 0.7229 - recall: 0.7103 - val_loss: 0.5338 - val_accuracy: 0.7487 - val_recall: 0.8673
Epoch 2/5
43/43 [==============================] - 79s 2s/step - loss: 0.5138 - accuracy: 0.7593 - recall: 0.7207 - val_loss: 0.4887 - val_accuracy: 0.7808 - val_recall: 0.7585
Epoch 3/5
43/43 [==============================] - 78s 2s/step - loss: 0.4982 - accuracy: 0.7727 - recall: 0.7028 - val_loss: 0.5109 - val_accuracy: 0.7572 - val_recall: 0.8333
Epoch 4/5
43/43 [==============================] - 78s 2s/step - loss: 0.4798 - accuracy: 0.7719 - recall: 0.7251 - val_loss: 0.4772 - val_accuracy: 0.7875 - val_recall: 0.7755
Epoch 5/5
43/43 [==============================] - 78s 2s/step - loss: 0.4652 - accuracy: 0.7912 - recall: 0.7623 - val_loss: 0.4864 - val_accuracy: 0.7791 - val_recall: 0.7211
Total Time Elapsed:  6  minutes  41  seconds


In [7]:
scores = model.evaluate(test_g, verbose=1)

30/30 [==============================] - 14s 475ms/step - loss: 0.9313 - accuracy: 0.4879 - recall: 0.2855


## 2x2x1C1D

Try deeper structure.

In [4]:
model = models.Sequential()
model.add(layers.Conv2D(64, 3, activation = 'relu', input_shape=(256,256,3)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['accuracy', 'Recall'])

start = timer()

history = model.fit(train_g, 
                    epochs=5, 
                    validation_data=val_g, 
                    workers = 7
                   )

end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/5
43/43 [==============================] - 283s 7s/step - loss: 0.5827 - accuracy: 0.7325 - recall: 0.6612 - val_loss: 0.5210 - val_accuracy: 0.7487 - val_recall: 0.8061
Epoch 2/5
43/43 [==============================] - 266s 6s/step - loss: 0.5241 - accuracy: 0.7578 - recall: 0.7177 - val_loss: 0.5490 - val_accuracy: 0.7521 - val_recall: 0.8810
Epoch 3/5
43/43 [==============================] - 266s 6s/step - loss: 0.5061 - accuracy: 0.7682 - recall: 0.7311 - val_loss: 0.5380 - val_accuracy: 0.7622 - val_recall: 0.8605
Epoch 4/5
43/43 [==============================] - 265s 6s/step - loss: 0.4951 - accuracy: 0.7697 - recall: 0.7281 - val_loss: 0.5025 - val_accuracy: 0.7605 - val_recall: 0.7925
Epoch 5/5
43/43 [==============================] - 265s 6s/step - loss: 0.4921 - accuracy: 0.7741 - recall: 0.7251 - val_loss: 0.5079 - val_accuracy: 0.7757 - val_recall: 0.7687
Total Time Elapsed:  22  minutes  55  seconds


In [5]:
scores = model.evaluate(test_g, verbose=1)

30/30 [==============================] - 37s 1s/step - loss: 0.8040 - accuracy: 0.4857 - recall: 0.2901


## 2x2x1C2D

Add Dense layer.

In [6]:
model = models.Sequential()
model.add(layers.Conv2D(64, 3, activation = 'relu', input_shape=(256,256,3)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['accuracy', 'Recall'])

start = timer()

history = model.fit(train_g, 
                    epochs=5, 
                    validation_data=val_g, 
                    workers = 7
                   )

end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/5
43/43 [==============================] - 254s 6s/step - loss: 0.5606 - accuracy: 0.7281 - recall: 0.7132 - val_loss: 0.5072 - val_accuracy: 0.7673 - val_recall: 0.7211
Epoch 2/5
43/43 [==============================] - 240s 6s/step - loss: 0.5149 - accuracy: 0.7608 - recall: 0.7132 - val_loss: 0.5200 - val_accuracy: 0.7622 - val_recall: 0.8401
Epoch 3/5
43/43 [==============================] - 240s 6s/step - loss: 0.4988 - accuracy: 0.7808 - recall: 0.7355 - val_loss: 0.5376 - val_accuracy: 0.7622 - val_recall: 0.8741
Epoch 4/5
43/43 [==============================] - 243s 6s/step - loss: 0.4946 - accuracy: 0.7741 - recall: 0.7400 - val_loss: 0.5496 - val_accuracy: 0.7521 - val_recall: 0.8673
Epoch 5/5
43/43 [==============================] - 265s 6s/step - loss: 0.4794 - accuracy: 0.7838 - recall: 0.7444 - val_loss: 0.5064 - val_accuracy: 0.7605 - val_recall: 0.7857
Total Time Elapsed:  21  minutes  11  seconds


In [7]:
scores = model.evaluate(test_g, verbose=1)

30/30 [==============================] - 37s 1s/step - loss: 0.8977 - accuracy: 0.5259 - recall: 0.3688


## 2x2x1x1C1D

Try deeper structure with another layer.

In [8]:
model = models.Sequential()
model.add(layers.Conv2D(64, 3, activation = 'relu', input_shape=(256,256,3)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['accuracy', 'Recall'])

start = timer()

history = model.fit(train_g, 
                    epochs=5, 
                    validation_data=val_g, 
                    workers = 7
                   )

end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/5
43/43 [==============================] - 270s 6s/step - loss: 0.5692 - accuracy: 0.7244 - recall: 0.6924 - val_loss: 0.6684 - val_accuracy: 0.6796 - val_recall: 0.9320
Epoch 2/5
43/43 [==============================] - 292s 7s/step - loss: 0.5274 - accuracy: 0.7563 - recall: 0.7355 - val_loss: 0.4963 - val_accuracy: 0.7605 - val_recall: 0.7925
Epoch 3/5
43/43 [==============================] - 293s 7s/step - loss: 0.5142 - accuracy: 0.7652 - recall: 0.7444 - val_loss: 0.4906 - val_accuracy: 0.7605 - val_recall: 0.7789
Epoch 4/5
43/43 [==============================] - 288s 7s/step - loss: 0.5025 - accuracy: 0.7712 - recall: 0.7296 - val_loss: 0.4999 - val_accuracy: 0.7723 - val_recall: 0.8129
Epoch 5/5
43/43 [==============================] - 286s 7s/step - loss: 0.4995 - accuracy: 0.7734 - recall: 0.7400 - val_loss: 0.4851 - val_accuracy: 0.7555 - val_recall: 0.7721
Total Time Elapsed:  24  minutes  22  seconds


In [9]:
scores = model.evaluate(test_g, verbose=1)

30/30 [==============================] - 42s 1s/step - loss: 0.9306 - accuracy: 0.5048 - recall: 0.3256


## 2x2x1x1C1D Adam

Try Adam optimizer instead of RMSprop for faster convergence.

In [4]:
model = models.Sequential()
model.add(layers.Conv2D(64, 3, activation = 'relu', input_shape=(256,256,3)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(),
              metrics=['accuracy', 'Recall'])

start = timer()

history = model.fit(train_g, 
                    epochs=5, 
                    validation_data=val_g, 
                    workers = 7
                   )

end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/5
43/43 [==============================] - 261s 6s/step - loss: 0.5660 - accuracy: 0.7214 - recall: 0.7088 - val_loss: 0.6083 - val_accuracy: 0.7369 - val_recall: 0.8537
Epoch 2/5
43/43 [==============================] - 268s 6s/step - loss: 0.5142 - accuracy: 0.7719 - recall: 0.7013 - val_loss: 0.5094 - val_accuracy: 0.7639 - val_recall: 0.7993
Epoch 3/5
43/43 [==============================] - 279s 6s/step - loss: 0.5071 - accuracy: 0.7645 - recall: 0.7088 - val_loss: 0.4937 - val_accuracy: 0.7757 - val_recall: 0.7551
Epoch 4/5
43/43 [==============================] - 266s 6s/step - loss: 0.4985 - accuracy: 0.7779 - recall: 0.7444 - val_loss: 0.4926 - val_accuracy: 0.7757 - val_recall: 0.7619
Epoch 5/5
43/43 [==============================] - 280s 7s/step - loss: 0.5133 - accuracy: 0.7608 - recall: 0.6909 - val_loss: 0.5149 - val_accuracy: 0.7572 - val_recall: 0.8265
Total Time Elapsed:  22  minutes  60  seconds


In [5]:
scores = model.evaluate(test_g, verbose=1)

30/30 [==============================] - 37s 1s/step - loss: 0.8501 - accuracy: 0.5248 - recall: 0.3981


## 2x2x1x1C1D Adam / BatchNorm

Add BatchNormalization for improved performance.

In [6]:
model = models.Sequential()
model.add(layers.Conv2D(96, 3, activation = 'relu', input_shape=(256,256,3)))
model.add(layers.Conv2D(96, 3, activation = 'relu'))
model.add(BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(32, 3, activation = 'relu'))
model.add(BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(),
              metrics=['accuracy'])

start = timer()

history = model.fit(train_g, 
                    epochs=5, 
                    validation_data=val_g, 
                    workers = 7
                   )

end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/5
43/43 [==============================] - 489s 11s/step - loss: 0.9327 - accuracy: 0.7043 - val_loss: 0.6597 - val_accuracy: 0.7116
Epoch 2/5
43/43 [==============================] - 535s 12s/step - loss: 0.5530 - accuracy: 0.7318 - val_loss: 0.6387 - val_accuracy: 0.6779
Epoch 3/5
43/43 [==============================] - 469s 11s/step - loss: 0.4851 - accuracy: 0.7801 - val_loss: 0.6176 - val_accuracy: 0.7791
Epoch 4/5
43/43 [==============================] - 450s 10s/step - loss: 0.4551 - accuracy: 0.7853 - val_loss: 0.6077 - val_accuracy: 0.6476
Epoch 5/5
43/43 [==============================] - 431s 10s/step - loss: 0.4457 - accuracy: 0.7912 - val_loss: 0.6177 - val_accuracy: 0.7740
Total Time Elapsed:  40  minutes  29  seconds


In [7]:
scores = model.evaluate(test_g, verbose=1)

30/30 [==============================] - 63s 2s/step - loss: 0.7235 - accuracy: 0.4678


## 1x1C1D Adam/BatchNorm

Try simpler model with Adam optimizer and Batch Normalization layers.

In [10]:
model = models.Sequential()
model.add(layers.Conv2D(96, 3, activation = 'relu', input_shape=(256,256,3)))
model.add(BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(),
              metrics=['accuracy', 'Recall'])

start = timer()

history = model.fit(train_g, 
                    epochs=5, 
                    validation_data=val_g, 
                    workers = 7
                   )

end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/5
43/43 [==============================] - 181s 4s/step - loss: 16.3543 - accuracy: 0.6419 - recall: 0.6597 - val_loss: 2.5317 - val_accuracy: 0.5042 - val_recall: 0.0000e+00
Epoch 2/5
43/43 [==============================] - 187s 4s/step - loss: 3.0000 - accuracy: 0.7080 - recall: 0.6880 - val_loss: 1.9641 - val_accuracy: 0.5042 - val_recall: 0.0000e+00
Epoch 3/5
43/43 [==============================] - 184s 4s/step - loss: 2.2222 - accuracy: 0.7415 - recall: 0.7429 - val_loss: 0.9555 - val_accuracy: 0.5042 - val_recall: 0.0000e+00
Epoch 4/5
43/43 [==============================] - 184s 4s/step - loss: 1.2783 - accuracy: 0.7719 - recall: 0.7786 - val_loss: 0.6944 - val_accuracy: 0.5025 - val_recall: 0.0340
Epoch 5/5
43/43 [==============================] - 172s 4s/step - loss: 2.1308 - accuracy: 0.7615 - recall: 0.7756 - val_loss: 0.8881 - val_accuracy: 0.5042 - val_recall: 0.0000e+00
Total Time Elapsed:  15  minutes  34  seconds


In [11]:
scores = model.evaluate(test_g, verbose=1)

30/30 [==============================] - 25s 843ms/step - loss: 1.1091 - accuracy: 0.3200 - recall: 0.0077


## 1x1C1D BatchNorm

Try Batch Normalization with RMSprop optimizer.

In [5]:
model = models.Sequential()
model.add(layers.Conv2D(96, 3, activation = 'relu', input_shape=(256,256,3)))
model.add(BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(),
              metrics=['accuracy', 'Recall'])

start = timer()

history = model.fit(train_g, 
                    epochs=5, 
                    validation_data=val_g, 
                    workers = 7
                   )

end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/5
43/43 [==============================] - 170s 4s/step - loss: 28.2369 - accuracy: 0.6508 - recall: 0.4889 - val_loss: 0.5945 - val_accuracy: 0.6442 - val_recall: 0.9082
Epoch 2/5
43/43 [==============================] - 182s 4s/step - loss: 1.0080 - accuracy: 0.6263 - recall: 0.3730 - val_loss: 0.6534 - val_accuracy: 0.5312 - val_recall: 0.9286
Epoch 3/5
43/43 [==============================] - 173s 4s/step - loss: 2.7178 - accuracy: 0.7288 - recall: 0.6969 - val_loss: 0.8279 - val_accuracy: 0.4992 - val_recall: 0.0204
Epoch 4/5
43/43 [==============================] - 174s 4s/step - loss: 0.5921 - accuracy: 0.7429 - recall: 0.6152 - val_loss: 2.9566 - val_accuracy: 0.5042 - val_recall: 0.0000e+00
Epoch 5/5
43/43 [==============================] - 172s 4s/step - loss: 0.7711 - accuracy: 0.7511 - recall: 0.6478 - val_loss: 12.3006 - val_accuracy: 0.5042 - val_recall: 0.0000e+00
Total Time Elapsed:  14  minutes  53  seconds


In [6]:
scores = model.evaluate(test_g, verbose=1)

30/30 [==============================] - 22s 718ms/step - loss: 17.4486 - accuracy: 0.3157 - recall: 0.0000e+00


## 4x1C1D

Try even deeper structure with 4 individual layers.

In [7]:
model = models.Sequential()
model.add(layers.Conv2D(96, 3, activation = 'relu', input_shape=(256,256,3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(),
              metrics=['accuracy'])

start = timer()

history = model.fit(train_g, 
                    epochs=5, 
                    validation_data=val_g, 
                    workers = 7
                   )

end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/5
43/43 [==============================] - 108s 3s/step - loss: 0.6059 - accuracy: 0.7021 - val_loss: 0.5018 - val_accuracy: 0.7622
Epoch 2/5
43/43 [==============================] - 109s 3s/step - loss: 0.5281 - accuracy: 0.7637 - val_loss: 0.5465 - val_accuracy: 0.7622
Epoch 3/5
43/43 [==============================] - 111s 3s/step - loss: 0.5134 - accuracy: 0.7652 - val_loss: 0.4828 - val_accuracy: 0.7622
Epoch 4/5
43/43 [==============================] - 110s 3s/step - loss: 0.5010 - accuracy: 0.7741 - val_loss: 0.4880 - val_accuracy: 0.7825
Epoch 5/5
43/43 [==============================] - 111s 3s/step - loss: 0.4877 - accuracy: 0.7838 - val_loss: 0.5114 - val_accuracy: 0.7673
Total Time Elapsed:  9  minutes  23  seconds


In [8]:
scores = model.evaluate(test_g, verbose=1)

30/30 [==============================] - 20s 679ms/step - loss: 0.7397 - accuracy: 0.4995


## 3x1C3D

Try deeper Dense layers.

In [9]:
model = models.Sequential()
model.add(layers.Conv2D(96, 3, activation = 'relu', input_shape=(256,256,3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(128, activation = 'relu'))
model.add(layers.Dense(64, activation = 'relu'))
model.add(layers.Dense(32, activation = 'relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(),
              metrics=['accuracy', 'Recall'])

start = timer()

history = model.fit(train_g, 
                    epochs=5, 
                    validation_data=val_g, 
                    workers = 7
                   )

end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/5
43/43 [==============================] - 123s 3s/step - loss: 0.6837 - accuracy: 0.7021 - recall: 0.6538 - val_loss: 0.5049 - val_accuracy: 0.7572 - val_recall: 0.8401
Epoch 2/5
43/43 [==============================] - 119s 3s/step - loss: 0.5155 - accuracy: 0.7615 - recall: 0.7221 - val_loss: 0.6927 - val_accuracy: 0.7066 - val_recall: 0.9320
Epoch 3/5
43/43 [==============================] - 124s 3s/step - loss: 0.5002 - accuracy: 0.7712 - recall: 0.7311 - val_loss: 0.5026 - val_accuracy: 0.7622 - val_recall: 0.7721
Epoch 4/5
43/43 [==============================] - 123s 3s/step - loss: 0.4865 - accuracy: 0.7845 - recall: 0.7563 - val_loss: 0.4912 - val_accuracy: 0.7926 - val_recall: 0.7551
Epoch 5/5
43/43 [==============================] - 122s 3s/step - loss: 0.4648 - accuracy: 0.7905 - recall: 0.7637 - val_loss: 0.5063 - val_accuracy: 0.7943 - val_recall: 0.7483
Total Time Elapsed:  10  minutes  27  seconds


In [10]:
scores = model.evaluate(test_g, verbose=1)

30/30 [==============================] - 16s 549ms/step - loss: 1.3665 - accuracy: 0.4678 - recall: 0.2500


## 2C3D

Try deeper Dense layers.

In [9]:
model = models.Sequential()
model.add(layers.Conv2D(96, 3, activation = 'relu', input_shape=(256,256,3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(128, activation = 'relu'))
model.add(layers.Dense(64, activation = 'relu'))
model.add(layers.Dense(32, activation = 'relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(),
              metrics=['accuracy', 'Recall'])

start = timer()

history = model.fit(train_g, 
                    epochs=5, 
                    validation_data=val_g, 
                    workers = 7
                   )

end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/5
43/43 [==============================] - 106s 2s/step - loss: 0.6466 - accuracy: 0.7266 - recall: 0.6508 - val_loss: 0.5084 - val_accuracy: 0.7589 - val_recall: 0.7449
Epoch 2/5
43/43 [==============================] - 107s 2s/step - loss: 0.5236 - accuracy: 0.7645 - recall: 0.7251 - val_loss: 0.4848 - val_accuracy: 0.7723 - val_recall: 0.7653
Epoch 3/5
43/43 [==============================] - 109s 3s/step - loss: 0.5049 - accuracy: 0.7808 - recall: 0.7533 - val_loss: 0.5145 - val_accuracy: 0.7774 - val_recall: 0.8061
Epoch 4/5
43/43 [==============================] - 107s 2s/step - loss: 0.4295 - accuracy: 0.8143 - recall: 0.7920 - val_loss: 0.5047 - val_accuracy: 0.7791 - val_recall: 0.7789
Epoch 5/5
43/43 [==============================] - 107s 2s/step - loss: 0.3768 - accuracy: 0.8284 - recall: 0.8172 - val_loss: 0.5594 - val_accuracy: 0.7336 - val_recall: 0.8707
Total Time Elapsed:  9  minutes  10  seconds


In [10]:
scores = model.evaluate(test_g, verbose=1)

30/30 [==============================] - 18s 593ms/step - loss: 0.9085 - accuracy: 0.5438 - recall: 0.4738


## 2x2C1D Dropout

Try adding Dropout to combat overfitting.

In [5]:
model = models.Sequential()
model.add(layers.Conv2D(64, 3, activation = 'relu', input_shape=(256,256,3)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation = 'relu'))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(),
              metrics=['accuracy', 'Recall'])

start = timer()

history = model.fit(train_g, 
                    epochs=8, 
                    validation_data=val_g, 
                    workers = 7
                   )

end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/8
43/43 [==============================] - 237s 6s/step - loss: 0.8407 - accuracy: 0.6538 - recall: 0.5513 - val_loss: 0.5106 - val_accuracy: 0.7538 - val_recall: 0.7789
Epoch 2/8
43/43 [==============================] - 235s 5s/step - loss: 0.5510 - accuracy: 0.7511 - recall: 0.7028 - val_loss: 0.4977 - val_accuracy: 0.7656 - val_recall: 0.7755
Epoch 3/8
43/43 [==============================] - 236s 5s/step - loss: 0.5303 - accuracy: 0.7519 - recall: 0.7177 - val_loss: 1.6961 - val_accuracy: 0.4975 - val_recall: 1.0000
Epoch 4/8
43/43 [==============================] - 236s 5s/step - loss: 0.5350 - accuracy: 0.7697 - recall: 0.7325 - val_loss: 0.5011 - val_accuracy: 0.7707 - val_recall: 0.7347
Epoch 5/8
43/43 [==============================] - 236s 5s/step - loss: 0.4736 - accuracy: 0.7675 - recall: 0.7236 - val_loss: 0.5086 - val_accuracy: 0.7740 - val_recall: 0.7823
Epoch 6/8
43/43 [==============================] - 236s 5s/step - loss: 0.4471 - accuracy: 0.8068 - recall: 0.

In [7]:
scores = model.evaluate(test_g, verbose=1)

19/19 [==============================] - 18s 974ms/step - loss: 0.7373 - accuracy: 0.5767 - recall: 0.9592


## 2x1x1C1D Dropout

Add Dropout to model to combat overfitting.

In [11]:
model = models.Sequential()
model.add(layers.Conv2D(64, 3, activation = 'relu', input_shape=(256,256,3)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))

model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))

model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))

model.add(layers.Flatten())
model.add(layers.Dense(128, activation = 'relu'))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(),
              metrics=['accuracy', 'Recall'])

start = timer()

history = model.fit(train_g, 
                    epochs=10, 
                    validation_data=val_g, 
                    workers = 7
                   )

end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/10
43/43 [==============================] - 235s 5s/step - loss: 0.6728 - accuracy: 0.6768 - recall: 0.6062 - val_loss: 0.5184 - val_accuracy: 0.7555 - val_recall: 0.8027
Epoch 2/10
43/43 [==============================] - 240s 6s/step - loss: 0.5272 - accuracy: 0.7533 - recall: 0.6954 - val_loss: 0.5191 - val_accuracy: 0.7555 - val_recall: 0.8469
Epoch 3/10
43/43 [==============================] - 236s 5s/step - loss: 0.5182 - accuracy: 0.7667 - recall: 0.7177 - val_loss: 0.5123 - val_accuracy: 0.7639 - val_recall: 0.8163
Epoch 4/10
43/43 [==============================] - 234s 5s/step - loss: 0.4931 - accuracy: 0.7630 - recall: 0.7385 - val_loss: 0.5189 - val_accuracy: 0.7656 - val_recall: 0.7891
Epoch 5/10
43/43 [==============================] - 234s 5s/step - loss: 0.4833 - accuracy: 0.7749 - recall: 0.7340 - val_loss: 0.5025 - val_accuracy: 0.7589 - val_recall: 0.7823
Epoch 6/10
43/43 [==============================] - 230s 5s/step - loss: 0.4695 - accuracy: 0.7853 - reca

In [12]:
scores = model.evaluate(test_g, verbose=1)

30/30 [==============================] - 35s 1s/step - loss: 0.7906 - accuracy: 0.5544 - recall: 0.4969


## 2x1x1C1D Dropout Adam

Try Dropout and Adam optimizer.

In [4]:
model = models.Sequential()
model.add(layers.Conv2D(64, 3, activation = 'relu', input_shape=(256,256,3)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))

model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))

model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))

model.add(layers.Flatten())
model.add(layers.Dense(128, activation = 'relu'))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(),
              metrics=['accuracy', 'Recall'])

start = timer()

history = model.fit(train_g, 
                    epochs=10, 
                    validation_data=val_g, 
                    workers = 7
                   )

end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/10
43/43 [==============================] - 259s 6s/step - loss: 0.5778 - accuracy: 0.7288 - recall: 0.6597 - val_loss: 0.5040 - val_accuracy: 0.7555 - val_recall: 0.7721
Epoch 2/10
43/43 [==============================] - 231s 5s/step - loss: 0.5176 - accuracy: 0.7779 - recall: 0.7355 - val_loss: 0.5605 - val_accuracy: 0.7622 - val_recall: 0.8095
Epoch 3/10
43/43 [==============================] - 225s 5s/step - loss: 0.4981 - accuracy: 0.7697 - recall: 0.7415 - val_loss: 0.5237 - val_accuracy: 0.7622 - val_recall: 0.7823
Epoch 4/10
43/43 [==============================] - 223s 5s/step - loss: 0.4900 - accuracy: 0.7823 - recall: 0.7474 - val_loss: 0.5216 - val_accuracy: 0.7639 - val_recall: 0.7823
Epoch 5/10
43/43 [==============================] - 226s 5s/step - loss: 0.4560 - accuracy: 0.7927 - recall: 0.7533 - val_loss: 0.5130 - val_accuracy: 0.7555 - val_recall: 0.8435
Epoch 6/10
43/43 [==============================] - 224s 5s/step - loss: 0.4495 - accuracy: 0.8031 - reca

In [5]:
scores = model.evaluate(test_g, verbose=1)

30/30 [==============================] - 35s 1s/step - loss: 1.1240 - accuracy: 0.4625 - recall: 0.2809


## 3C1D -- Fewer parameters

Tried fewer numbers of filters for Conv2D layers.

In [6]:
model = models.Sequential()
model.add(layers.Conv2D(32, 3, activation = 'relu', input_shape=(256,256,3)))
model.add(layers.Conv2D(32, 3, activation = 'relu'))
model.add(layers.Conv2D(32, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(128, activation = 'relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(),
              metrics=['accuracy', 'Recall'])

start = timer()

history = model.fit(train_g, 
                    epochs=5, 
                    validation_data=val_g, 
                    workers = 7
                   )

end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/5
43/43 [==============================] - 132s 3s/step - loss: 0.8183 - accuracy: 0.7028 - recall: 0.6776 - val_loss: 0.7067 - val_accuracy: 0.7201 - val_recall: 0.9252
Epoch 2/5
43/43 [==============================] - 131s 3s/step - loss: 0.5054 - accuracy: 0.7741 - recall: 0.7013 - val_loss: 0.5106 - val_accuracy: 0.7521 - val_recall: 0.8673
Epoch 3/5
43/43 [==============================] - 133s 3s/step - loss: 0.4546 - accuracy: 0.7890 - recall: 0.7593 - val_loss: 0.5212 - val_accuracy: 0.7605 - val_recall: 0.8095
Epoch 4/5
43/43 [==============================] - 128s 3s/step - loss: 0.3817 - accuracy: 0.8180 - recall: 0.7771 - val_loss: 0.6056 - val_accuracy: 0.7555 - val_recall: 0.8265
Epoch 5/5
43/43 [==============================] - 130s 3s/step - loss: 0.3137 - accuracy: 0.8700 - recall: 0.8455 - val_loss: 0.6016 - val_accuracy: 0.7504 - val_recall: 0.7755
Total Time Elapsed:  11  minutes  9  seconds


In [7]:
scores = model.evaluate(test_g, verbose=1)

30/30 [==============================] - 17s 570ms/step - loss: 1.3274 - accuracy: 0.4931 - recall: 0.3441


## Final Model Summary - 2x2C1D Dropout

In [5]:
model = models.Sequential()
model.add(layers.Conv2D(64, 3, activation = 'relu', input_shape=(256,256,3)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation = 'relu'))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(),
              metrics=['accuracy', 'Recall'])


In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 254, 254, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 252, 252, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 126, 126, 64)      0         
_________________________________________________________________
dropout (Dropout)            (None, 126, 126, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 124, 124, 64)      36928     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 122, 122, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 61, 61, 64)        0